# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Step 1: Import the sqlite3 module and pandas
import pandas as pd
import sqlite3

# Step 2: Connect to the database 'data.sqlite'
conn = sqlite3.connect('data.sqlite')

# Step 3: Create a cursor object to interact with the database
cur = conn.cursor()

# Optional: Print confirmation message
print("Connected to the database successfully.")

Connected to the database successfully.


In [6]:
# Step 4: Execute a query to retrieve the table names
q = ("""SELECT name 
                FROM sqlite_master 
                WHERE type='table';
""")
pd.read_sql(q, conn)
# Step 5: Fetch all results
#tables = cur.fetchall()

# Step 6: Print the table names
#for table in tables:
    #print(table[0])

,name
0,productlines
1,offices
2,customers
3,employees
4,orderdetails
5,orders
6,payments
7,products


In [4]:
# List of table names
tables = ['productlines', 'offices', 'customers', 'employees', 'orderdetails', 'orders', 'payments', 'products']

#Loop through each table and get column names
for table in tables:
    print(f"Columns in {table}:")
    
    # Execute PRAGMA statement to get column information
    cur.execute(f"PRAGMA table_info({table});")
    
    # Fetch all column information
    columns = cur.fetchall()
    
    # Print each column name
    for column in columns:
        print(column[1])  # The second item (index 1) is the column name
    
    print()  # Blank line for readability

Columns in productlines:
productLine
textDescription
htmlDescription
image

Columns in offices:
officeCode
city
phone
addressLine1
addressLine2
state
country
postalCode
territory

Columns in customers:
customerNumber
customerName
contactLastName
contactFirstName
phone
addressLine1
addressLine2
city
state
postalCode
country
salesRepEmployeeNumber
creditLimit

Columns in employees:
employeeNumber
lastName
firstName
extension
email
officeCode
reportsTo
jobTitle

Columns in orderdetails:
orderNumber
productCode
quantityOrdered
priceEach
orderLineNumber

Columns in orders:
orderNumber
orderDate
requiredDate
shippedDate
status
comments
customerNumber

Columns in payments:
customerNumber
checkNumber
paymentDate
amount

Columns in products:
productCode
productName
productLine
productScale
productVendor
productDescription
quantityInStock
buyPrice
MSRP



## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [8]:
q=("""SELECT employees.firstName, employees.lastName, offices.city
            FROM employees
            JOIN offices 
            ON employees.officeCode = offices.officeCode
            WHERE offices.city = 'Boston';
""")

pd.read_sql(q, conn)

# Step 4: Fetch all results
#employees_in_boston = cursor.fetchall()

# Step 5: Create a DataFrame
#df = pd.DataFrame(employees_in_boston, columns=['First Name', 'Last Name'])

# Step 6: Display the DataFrame
#print(df)

,firstName,lastName,city
0,Julie,Firrelli,Boston
1,Steve,Patterson,Boston


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [9]:
q = ("""SELECT offices.officeCode, offices.city, COUNT(employees.employeeNumber) AS num_employees
                FROM offices
                LEFT JOIN employees 
                ON offices.officeCode = employees.officeCode
                GROUP BY offices.officeCode, offices.city
                HAVING num_employees = 0;
""")
pd.read_sql(q, conn)

,officeCode,city,num_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [10]:
# Answers will vary
# Example question: 
"""
How many customers are there per office?
"""
q = ("""
SELECT offices.city,  COUNT(customers.city) AS num_customers
        FROM offices
        LEFT JOIN  customers 
        ON offices.city = customers.city
""")

pd.read_sql(q, conn)

,city,num_customers
0,San Francisco,16


In [11]:
q = ("""
SELECT offices.officeCode, offices.city, COUNT(customers.customerNumber) AS num_customers
        FROM offices
        JOIN employees ON offices.officeCode = employees.officeCode
        JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
        GROUP BY offices.officeCode, offices.city;
""")

pd.read_sql(q, conn)

,officeCode,city,num_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


# Question 1: How many employees are working in each office?
Answer: To find the number of employees working in each office, you can use a JOIN between the employees and offices tables and count the employees grouped by office.

In [12]:
q = ("""
SELECT offices.officeCode, offices.city, COUNT(employees.employeeNumber) AS num_employees
        FROM offices
        JOIN employees ON offices.officeCode = employees.officeCode
        GROUP BY offices.officeCode, offices.city; 
""")

pd.read_sql(q, conn)

,officeCode,city,num_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


# Question 2: Which customers have placed more than 5 orders?
Answer: To find customers who have placed more than 5 orders, you can join the customers and orders tables, group by customer, and use the HAVING clause to filter those with more than 5 orders.

In [15]:
q = ("""
SELECT customers.customerName, COUNT(orders.orderNumber) AS num_orders
        FROM customers
        JOIN orders ON customers.customerNumber = orders.customerNumber
        GROUP BY customers.customerName
        HAVING num_orders > 5;
""")

pd.read_sql(q, conn)

,customerName,num_orders
0,Euro+ Shopping Channel,26
1,Mini Gifts Distributors Ltd.,17


In [16]:
conn.close()

# Question 3: What is the total amount of payments made by each customer?
Answer: To calculate the total amount of payments made by each customer, you can join the customers and payments tables and sum the payment amounts.

In [ ]:
"""
Question 3
"""

q = ("""
        
""")

pd.read_sql(q, conn)

# Question 4: What are the details of orders that contain products from the 'Motorcycles' product line?
Answer: To find orders containing products from the 'Motorcycles' product line, you need to join orders, orderdetails, products, and productlines tables and filter for the 'Motorcycles' product line.

In [ ]:
q = ("""
        
""")

pd.read_sql(q, conn)

## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [ ]:
# Your code here

## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [ ]:
# Your code here

## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [ ]:
# Your code here

## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!